In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib inline

读取照片

In [2]:
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
# 数据增强
data_transforms = {
    'Train': transforms.Compose([
        # Can add more
        transforms.CenterCrop(224), # center crop 224*224 for resnet
        transforms.RandomHorizontalFlip(p=0.5), # Random horizontal rollover with a probability of 0.5
        transforms.RandomRotation(45), # turn -45°-45°
        transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1), # 亮度，对比度，饱和度，色相
        transforms.RandomGrayscale(p=0.025),
        transforms.ToTensor(), # Transfer to tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # Treatment of mean and standard deviation
    ]),
    'Test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(p=0.5), # Random horizontal rollover with a probability of 0.5
        transforms.RandomRotation(45), # 随机旋转-45°到45°
        transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.1), # 亮度，对比度，饱和度，色相
        transforms.RandomGrayscale(p=0.025),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}
# 训练集
# 在这里改数据源文件夹
data_dir = "New Masks Dataset"
img_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['Train', 'Test']}
dataloaders = {x: torch.utils.data.DataLoader(img_datasets[x], batch_size=256,
                                              shuffle=True)
               for x in ['Train', 'Test']}
class_names = img_datasets['Train'].classes

In [3]:
model_vgg = torchvision.models.vgg11(weights=torchvision.models.VGG11_Weights.DEFAULT)
for param in model_vgg.parameters():
    param.requires_grad = False
model_vgg.classifier._modules['6'] = nn.Linear(4096, 2)
model_vgg

Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to C:\Users\Lampard/.cache\torch\hub\checkpoints\vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [4]:
model_alex = torchvision.models.alexnet(weights=torchvision.models.AlexNet_Weights.DEFAULT)
for param in model_alex.parameters():
    param.requires_grad = False
model_alex.classifier._modules['6'] = nn.Linear(4096, 2)
model_alex

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\Lampard/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

优化器

In [5]:
optimizer_ft = optim.Adam(model_vgg.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()
optimizer_f = optim.Adam(model_alex.parameters(), lr=0.0001)

训练

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        _, out_pred = torch.max(model(X), 1)

        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 2 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

            # 看一下，以后再删
            # print("predict", out_pred)
            # print("target", y)

def test_loop(dataloader, model, loss_fn, acc_list, loss_list):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    acc_list.append(correct*100)
    loss_list.append(test_loss)
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
def draw_acc_result(acc, epoch_time):
    y = np.array(acc)
    x = np.array(epoch_time)
    plt.plot(x, y)

    plt.title('Accuracy Epoch Line Chart')
    plt.ylabel('y-accuracy')
    plt.xlabel('x-epoch')
    plt.show()
def draw_loss_result(loss, epoch_time):
    y = np.array(loss)
    x = np.array(epoch_time)
    plt.plot(x, y)

    plt.title('Loss Epoch Line Chart')
    plt.ylabel('y-loss')
    plt.xlabel('x-epoch')
    plt.show()

测试vgg

In [ ]:
epochs = 50
acc_list = []
loss_list = []
if torch.cuda.is_available():
    print("Start on GPU")
else:
    print("Start on CPU")
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloaders["Train"], model_vgg, loss_fn, optimizer_ft)
    test_loop(dataloaders["Test"], model_vgg, loss_fn, acc_list, loss_list)
print("Done!")
draw_acc_result(acc_list, [i for i in range(1, len(acc_list)+1)])
draw_loss_result(loss_list, [i for i in range(1, len(loss_list)+1)])

Start on CPU
Epoch 1
-------------------------------
loss: 0.710480  [    0/ 2000]
loss: 0.676840  [  512/ 2000]
loss: 0.640044  [ 1024/ 2000]
loss: 0.646550  [ 1536/ 2000]
Test Error: 
 Accuracy: 69.0%, Avg loss: 0.599772 

Epoch 2
-------------------------------
loss: 0.629037  [    0/ 2000]
loss: 0.585401  [  512/ 2000]
loss: 0.543369  [ 1024/ 2000]
loss: 0.515258  [ 1536/ 2000]
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.456675 

Epoch 3
-------------------------------
loss: 0.492027  [    0/ 2000]
loss: 0.489538  [  512/ 2000]
loss: 0.461497  [ 1024/ 2000]
loss: 0.465660  [ 1536/ 2000]
Test Error: 
 Accuracy: 89.0%, Avg loss: 0.414919 

Epoch 4
-------------------------------
loss: 0.434130  [    0/ 2000]
loss: 0.449782  [  512/ 2000]
loss: 0.431494  [ 1024/ 2000]
loss: 0.405919  [ 1536/ 2000]
Test Error: 
 Accuracy: 89.5%, Avg loss: 0.374764 

Epoch 5
-------------------------------
loss: 0.374359  [    0/ 2000]
loss: 0.362174  [  512/ 2000]
loss: 0.372435  [ 1024/ 2000]
loss: 0.3

测试 alexnet

In [ ]:
epochs = 50
acc_list = []
loss_list = []
if torch.cuda.is_available():
    print("Start on GPU")
else:
    print("Start on CPU")
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(dataloaders["Train"], model_alex, loss_fn, optimizer_ft)
    test_loop(dataloaders["Test"], model_alex, loss_fn, acc_list, loss_list)
print("Done!")
draw_acc_result(acc_list, [i for i in range(1, len(acc_list)+1)])
draw_loss_result(loss_list, [i for i in range(1, len(loss_list)+1)])